In [4]:
import requests
from bs4 import BeautifulSoup
import re
import urllib
import json
from datetime import datetime
import dateutil.parser
from bson import json_util


In [5]:
#"مادة ويكي بيانات"
# That's the "Wikidata" link

In [6]:
list_link = "https://en.wikipedia.org/wiki/Category:Government_ministers_of_Somalia"

In [3]:
person = "https://en.wikipedia.org/wiki/John_Kerry"
#person = "https://ar.wikipedia.org/wiki/%D8%AC%D9%88%D9%86_%D9%83%D9%8A%D8%B1%D9%8A"

In [7]:
def contains_month(text):
    months_en = ['january ', 'february ', 'march ', 'april ', 'may ', 'june ', 
                  'july ', 'august ', 'september ', 'october ', 'november ', 'december ']
    months_ar = ["إبريل","يناير", "فبراير", "مارس", "أبريل", "مايو", "يونية", "يونيو",
                "يولية", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر", 
                "يناير"]
    for month in months_en:
        if month in text.lower():
            return True
    return False

def get_dates(i, results):
    dates = []
    while i < len(results):
        results[i] = results[i].replace('\xa0', '')
        # check if date
        index = results[i].find(' ')
        if index == -1:
            break
        elif not contains_month(results[i]):
            break
        date_range = results[i]
        if '–' in date_range:
            date_range = date_range.split('–')
            dates.append((date_range[0], date_range[1]))
        else:
            dates.append((date_range, None))
        i += 1
    return (dates, i)

def format_wiki_links(lang):
    lang_code = lang.find("a")["hreflang"]
    wiki_link = lang.find(href=True)['href']
    name = re.findall("wiki/(.+)", wiki_link)[0]
    name = re.sub("_", " ", name)
    name = urllib.parse.unquote(name)

    return {lang_code : {"name" : name,
                        "wiki_link" : wiki_link}}

def get_wiki_links(soup):
    langs = soup.find_all("li", {"class" : "interlanguage-link"})
    alt_names = []
    for i in langs:
        t = format_wiki_links(i)
        alt_names.append(t)
    return alt_names


def get_roles(soup):
    table = soup.find('table', { 'class': 'infobox vcard' })     # <-- change for Arabic!
    potential = table.find_all(['th', 'td', {'colspan' : '2'}])  # <-- change for Arabic!
    results = []
    for elem in potential:
        if elem.name == 'th' and elem.has_attr('colspan'): # <-- change for Arabic!
            results.append(elem.text)
        elif '\n' in elem.text and 'office' in elem.text: # <-- change for Arabic!
            results.append(elem.text.splitlines()[1])
    results = results[1:]    # Discard name
    mapping = {}
    i = 0
    while i < len(results):
        office = results[i]
        i += 1
        (dates, i) = get_dates(i, results)
        mapping[office] = dates
    mapping.pop('Personal details', None)
    mapping.pop('Military service', None)

    roles = []
    for title in mapping.keys():
        for period in mapping[title]:
            if period[0] and period[1]:
                start_date = period[0]
                end_date = period[1]
            elif not period[1]:
                start_date = period[0]
                end_date = ""
            try:
                start_date = dateutil.parser.parse(start_date)
            except ValueError:
                start_date = None
            try:
                end_date = dateutil.parser.parse(end_date)
            except ValueError:
                end_date = None
            entry = {"title" : title,
                        "start_date" : start_date,
                        "end_date" : end_date}

            roles.append(entry)
    return roles

def get_wikidata_id(soup):
    wikidata = soup.find("li", {"id" : "t-wikibase"}).find(href=True)['href']
    wikidata_id = re.findall("Page/(.+)", wikidata)
    return wikidata_id

In [9]:
def page_to_data(page):
    """Only works for people probably, not orgs..."""
    p_page = requests.get(page)
    soup = BeautifulSoup(p_page.content, "lxml")
    wikidata_id = get_wikidata_id(soup)
    roles = get_roles(soup)
    wiki_pages = get_wiki_links(soup)


    entry = {"wiki_pages" : wiki_pages,
            "scrape_date" : str(datetime.now()),
            "wikidata_id" : wikidata_id,
            "wiki_roles" : {"en" : roles},
            "cameo_coding" : []
            }
    
    return entry
    

hadi = page_to_data("https://en.wikipedia.org/wiki/Abdrabbuh_Mansur_Hadi")

In [8]:
hadi

NameError: name 'hadi' is not defined

In [9]:
json.dumps(hadi, default=json_util.default)

'{"wiki_pages": [{"ar": {"name": "\\u0639\\u0628\\u062f \\u0631\\u0628\\u0647 \\u0645\\u0646\\u0635\\u0648\\u0631 \\u0647\\u0627\\u062f\\u064a", "wiki_link": "https://ar.wikipedia.org/wiki/%D8%B9%D8%A8%D8%AF_%D8%B1%D8%A8%D9%87_%D9%85%D9%86%D8%B5%D9%88%D8%B1_%D9%87%D8%A7%D8%AF%D9%8A"}}, {"azb": {"name": "\\u0639\\u0628\\u062f \\u0631\\u0628\\u0647 \\u0645\\u0646\\u0635\\u0648\\u0631 \\u0647\\u0627\\u062f\\u06cc", "wiki_link": "https://azb.wikipedia.org/wiki/%D8%B9%D8%A8%D8%AF_%D8%B1%D8%A8%D9%87_%D9%85%D9%86%D8%B5%D9%88%D8%B1_%D9%87%D8%A7%D8%AF%DB%8C"}}, {"bn": {"name": "\\u0986\\u09ac\\u09cd\\u09a6\\u09b0\\u09be\\u09ac\\u09cd\\u09ac\\u09c1\\u09b9 \\u09ae\\u09be\\u09a8\\u09b8\\u09c1\\u09b0 \\u09b9\\u09be\\u09a6\\u09bf", "wiki_link": "https://bn.wikipedia.org/wiki/%E0%A6%86%E0%A6%AC%E0%A7%8D%E0%A6%A6%E0%A6%B0%E0%A6%BE%E0%A6%AC%E0%A7%8D%E0%A6%AC%E0%A7%81%E0%A6%B9_%E0%A6%AE%E0%A6%BE%E0%A6%A8%E0%A6%B8%E0%A7%81%E0%A6%B0_%E0%A6%B9%E0%A6%BE%E0%A6%A6%E0%A6%BF"}}, {"be": {"name": "\\u0410\\u0431\

In [10]:
roles = BeautifulSoup.find_all("th", {"style" : "text-align:center;background:lavender"})
for r in roles:
    print(r.text)
    
dates = soup.find_all("td", {"style" : "text-align:center;border-bottom:none"})

date_list = []
for d in dates:
    dt = re.sub("In office", "", d.text).strip()
    date_list.append(dt)

date_list 

AttributeError: 'str' object has no attribute 'descendants'

In [14]:
# Some Arabic experimentation 
# (Not working yet)
table = BeautifulSoup.find('table', { 'class': 'infobox_v2 plainlist' })


#potential = table.find_all(['th', 'td', {'colspan' : '2'}])
potential = table.find_all(['tr', 'th', {'colspan' : '2'}])

results = []

# en
#for elem in potential:
#    if elem.name == 'th' and elem.has_attr('colspan'):
#        results.append(elem.text)
#    elif '\n' in elem.text and 'office' in elem.text:
#        results.append(elem.text.splitlines()[1])

for elem in potential:
    if elem.name == 'th' and elem.has_attr('colspan'):
        results.append(elem.text)
    elif '\n' in elem.text and 'في المنصب' in elem.text:
        results.append(elem.text.splitlines()[1])

results

AttributeError: 'str' object has no attribute 'find_all'

## Imagined final format

In [2]:
entry = {"names" : {"english" : [eng1, eng2, eng3],
                    "arabic" : [ar1, ar2],
                     "spanish" : [spa1, spa2],
                    "german" : [deu1]
                   }
        },
        "wiki_scrape_date" : today,
        "wikidata" : "Q10835103",
        "instance_of" : "person", # orgs and people look different
        "wikipedia" : {"english" : "/name_lastname",
                       "arabic" : "/name_lastname"},
         "harvested_from" : "http://wikipedia/category_list" # directory page we got the link from
         "wiki_categories" : {"english" : ["Living_Ministers", "Ministers_of_Somalia"],
                              "arabic" : []},
         "wiki_roles" : {"english" : [{"role" : "Minister of Irrigation",
                                      "start_date" : "2014-02-01",
                                      "end_date" : ""},
                                     ],
                         "arabic" : [{"role" : " وزارة الري",
                                      "start_date" : "2014-02-01",
                                      "end_date" : ""}]
                        },
         "cameo_coding" : [{"role" : "SOMGOV", 
                            "start_date" : "20140201", 
                            "end_date" : "",
                            "added_by" : "andy",
                            "added_date" : "2017-11-16"},
                          ]
        }

IndentationError: unexpected indent (<ipython-input-2-7453e4906352>, line 7)

In [31]:
# have different structures for people and organizations?
# With organizations, can skip "roles" and can add "commanders" or "members"
# Do we need to link the hand added CAMEO codings and the wiki roles?